In [ ]:
%matplotlib notebook

import cmapPy.pandasGEXpress.parse

import cmapPy.math.fast_corr

import plotly.express as pltexpr

import sklearn.cluster

import pandas as pd

import numpy

from matplotlib import pyplot as plt

from scipy.cluster.hierarchy import dendrogram

import scipy.cluster.hierarchy

import json

In [ ]:
gctoo = cmapPy.pandasGEXpress.parse.parse("2020_Q3_Achilles_CCLE_expression_r19144x1305.gctx")
print(gctoo)

In [ ]:
numpy_df = gctoo.data_df.to_numpy()

result = cmapPy.math.fast_corr.fast_corr(numpy_df)

In [ ]:
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = numpy.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = numpy.column_stack([model.children_, model.distances_,
                                      counts]).astype(float)
    
    print(linkage_matrix)

    # Plot the corresponding dendrogram
    r_dict = dendrogram(linkage_matrix, **kwargs)

    
    
    return linkage_matrix, list(map(int, r_dict["ivl"])), r_dict["leaves"]

# from https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_dendrogram.html#sphx-glr-auto-examples-cluster-plot-agglomerative-dendrogram-py

In [ ]:
fig, axs = plt.subplots(3,1)

In [ ]:
AffinityProp = sklearn.cluster.AffinityPropagation(affinity = "precomputed").fit(result)

print(AffinityProp)

In [ ]:
print(list(AffinityProp.labels_))

In [ ]:
result_df = pd.DataFrame(result)

labels_df = pd.DataFrame(AffinityProp.labels_)
labels_df.rename(columns = {0:1305}, inplace = True)

transposed_labels_df = labels_df.transpose()

transposed_labels_df.insert(1305, 1305, 20000)

print(result_df)
print(labels_df)
print(transposed_labels_df)

column_label_df = result_df.join(labels_df)

label_df = column_label_df.append(transposed_labels_df, ignore_index=True)

print(label_df)

col_sorted = label_df.sort_values(1305)

sorted_df = col_sorted.sort_values(1305, axis = 1)

print(col_sorted)
print(sorted_df)

sorted_df = sorted_df.drop(1305, axis = 0)
sorted_df = sorted_df.drop(1305, axis = 1)

print(sorted_df)



In [ ]:
cluster_centers = []

print(AffinityProp.cluster_centers_indices_)
for i in range (0, len(AffinityProp.cluster_centers_indices_)):
    cluster_centers.append(result[AffinityProp.cluster_centers_indices_[i]])

In [ ]:
ward_super_clusters = sklearn.cluster.AgglomerativeClustering(affinity = "euclidean", n_clusters = 71, linkage = "average", compute_full_tree = True, compute_distances = True).fit(cluster_centers)

In [ ]:
super_linkage_matrix, super_dendro_labels, super_leaves = plot_dendrogram(ward_super_clusters, truncate_mode='level', p=51, distance_sort = "ascending", ax=axs[0])
plt.show()

In [ ]:
print(super_dendro_labels)

super_dendro_labels_index = []

for i in range(0, len(super_dendro_labels)):
    super_dendro_labels_index.append(super_dendro_labels.index(i))
    

print(super_dendro_labels_index)
print(AffinityProp.cluster_centers_indices_)

In [ ]:
print(label_df)

#list comperhension 
#then add col


for i in range(0, 1306):
    val = label_df.loc[i, (1305)]
    if val != 20000:
        label_df.loc[i, (1305)] = super_dendro_labels_index[val]
        label_df.loc[1305, (i)] = super_dendro_labels_index[val]
    
    
    
super_col_sorted = label_df.sort_values(1305)

super_sorted_df = super_col_sorted.sort_values(1305, axis = 1)

print(super_col_sorted)
print(super_sorted_df)

with_labels_super_sorted_df = super_sorted_df

super_sorted_df = super_sorted_df.drop(1305, axis = 0)
super_sorted_df = super_sorted_df.drop(1305, axis = 1)

print(super_sorted_df)

In [ ]:
#df_super_linkage_matrix[4] == numpy.NaN
#current_sub_df = df_super_linkage_matrix.loc[df_super_linkage_matrix[0] == current_sub]
#if current_sub_df.shape[0] != 1:
    #current_sub_df = current_sub_df.loc[current_sub_df[4] == numpy.NaN]
    #if current_sub_df.shape[0] != 1:
        #current_sub_df = current_sub_df.loc[current_sub_df[5] == numpy.NaN]
    
#print(current_sub_df)



In [ ]:
df_super_linkage_matrix = pd.DataFrame(super_linkage_matrix) 

In [ ]:
print(df_super_linkage_matrix)

In [ ]:
 for i in range(0, len(df_super_linkage_matrix)):
        zero_leaf = df_super_linkage_matrix.loc[i, 0]
        one_leaf = df_super_linkage_matrix.loc[i, 1]
        if zero_leaf < len(super_dendro_labels_index):
            df_super_linkage_matrix.at[i, 0] = super_dendro_labels_index[int(zero_leaf)]
        if one_leaf < len(super_dendro_labels_index):
            df_super_linkage_matrix.at[i, 1] = super_dendro_labels_index[int(one_leaf)]

In [ ]:

df_super_linkage_matrix[4] = numpy.NaN
df_super_linkage_matrix[5] = numpy.NaN

#adding 4 to dist values of app super matrix 
df_super_linkage_matrix[2] = df_super_linkage_matrix[2] + 4

In [ ]:
print(df_super_linkage_matrix)

for row in df_super_linkage_matrix.iterrows():
    print(row)

In [ ]:
total_linkage_matrix = df_super_linkage_matrix

In [ ]:
sorted_with_labels = super_col_sorted.sort_values(1305, axis = 1)

In [ ]:
abnormal_dist = []

In [ ]:
def prepare_where(df_super_linkage_matrix, current_sub):
    try:
        where = df_super_linkage_matrix.loc[df_super_linkage_matrix[0] == current_sub]
        if where.shape[0] != 1:
            where = where.loc[where[4] == numpy.NaN]
            if where.shape[0] != 1:
                where = where.loc[where[5] == numpy.NaN]
        where_col = 0
        not_where = 1
        where.index[0]
        
    except IndexError:
        where = df_super_linkage_matrix.loc[df_super_linkage_matrix[1] == current_sub]
        if where.shape[0] != 1:
            where = where.loc[where[4] == numpy.NaN]
            if where.shape[0] != 1:
                where = where.loc[where[5] == numpy.NaN]
        where_col = 1
        not_where = 0
        
    if current_sub == 59:
        print("WHERE")
        print(where)
        
    return where, where_col, not_where

In [ ]:
#probably best to put all the handy values into a dict or something so I don't need to have a large amount of arguments

def prepare_sub_matrix(len_sub, df_sub_linkage_matrix, new_location, new_cluster, current_sub, len_super):
    for i in range(0, len_sub):
        zero_leaf = df_sub_linkage_matrix.loc[i, 0]
        one_leaf = df_sub_linkage_matrix.loc[i, 1]
        if zero_leaf >= n_sub_samples:
            df_sub_linkage_matrix.at[i, 0] = zero_leaf + len_super  + new_location

        else:
            df_sub_linkage_matrix[4][i] = sub_cluster.index[int(zero_leaf)]
            df_sub_linkage_matrix.at[i, 0] = new_cluster
            new_cluster += 1

        if one_leaf >= n_sub_samples:
            df_sub_linkage_matrix.at[i, 1] = one_leaf  + len_super + new_location


        else:
            df_sub_linkage_matrix[5][i] = sub_cluster.index[int(one_leaf)]
            df_sub_linkage_matrix.at[i, 1] = new_cluster
            new_cluster += 1

    #set the greatest value = to 1
    #that is the sample name that was replaced so this is done to aviod dupes
    
    for i in range(0, len(df_sub_linkage_matrix)):
        if df_sub_linkage_matrix.loc[i, 0] == len_super:
            df_sub_linkage_matrix.at[i, 0] = current_sub
            

        if df_sub_linkage_matrix.loc[i, 1] == len_super:
            df_sub_linkage_matrix.at[i, 1] = current_sub
        
    #change index to be index of where it is going
    df_sub_linkage_matrix.index = df_sub_linkage_matrix.index + new_location
    
    if current_sub == 59:
        print("df_sub_linkage_matrix")
        print(df_sub_linkage_matrix)
    
    return df_sub_linkage_matrix
    

In [ ]:
def prepare_super(len_super, df_super_linkage_matrix, len_sub, new_location):
        #change super cluster so sub cluster can be added 
    for i in range(0, len_super):
        
        zero_leaf = df_super_linkage_matrix.loc[i, 0]
        one_leaf = df_super_linkage_matrix.loc[i, 1]

        if zero_leaf >= n_super_samples:
            if zero_leaf - n_super_samples < new_location:
                df_super_linkage_matrix.at[i, 0] = df_super_linkage_matrix.loc[i, 0] + len_sub
            else:
                df_super_linkage_matrix.at[i, 0] = df_super_linkage_matrix.loc[i, 0] + (2 * len_sub) 

        if one_leaf >= n_super_samples:
            if one_leaf - n_super_samples < new_location:
                df_super_linkage_matrix.at[i, 1] = df_super_linkage_matrix.loc[i, 1] + len_sub
            else:
                df_super_linkage_matrix.at[i, 1] = df_super_linkage_matrix.loc[i, 1] + (2 * len_sub) 
                
    #drop where row and change indexs to prepare for append 
    df_super_linkage_matrix.drop(new_location, inplace = True)
    
    if current_sub == 59:
        print("df_super_linkage_matrix")
        print(df_super_linkage_matrix)
    
    return df_super_linkage_matrix

In [ ]:
for current_sub in range(0, len(super_dendro_labels_index)):
    #get all the values in the current sub cluster
    
    print("CURRENT SUB")
    print(current_sub)
    
    
    sub_cluster = sorted_with_labels.loc[sorted_with_labels[1305] == current_sub]
    
    # run ward
    ward_sub_cluster = sklearn.cluster.AgglomerativeClustering(affinity = "euclidean", n_clusters = len(sub_cluster)-1, linkage = "average", compute_full_tree = True, compute_distances = True).fit(sub_cluster)
    
    
    sub_linkage_matrix, sub_dendro_labels, sub_leaves = plot_dendrogram(ward_sub_cluster, truncate_mode='level', p=51, distance_sort = "ascending", ax= axs[1])
    plt.show()

    df_super_linkage_matrix = total_linkage_matrix 
    
    #make them dataframes since they are better to work with
    #df_super_linkage_matrix = pd.DataFrame(super_linkage_matrix) 
    #df_super_linkage_matrix[4] = numpy.NaN
    #df_super_linkage_matrix[5] = numpy.NaN

    
    #make them dataframes since they are better to work with
    df_sub_linkage_matrix = pd.DataFrame(sub_linkage_matrix) 
    
    #get where the sub cluster in in the super cluster matrix
    where_col = 5
    not_where = 5
    
    #and df_super_linkage_matrix[5] == -1

    where, where_col, not_where = prepare_where(df_super_linkage_matrix, current_sub)
    
    print("WHERE")
    print(where)
        
    
        
    #n_samples 
    n_super_samples = len(df_super_linkage_matrix) + 1
    n_sub_samples = len(df_sub_linkage_matrix) + 1
    
    len_super = len(df_super_linkage_matrix)
    len_sub = len(df_sub_linkage_matrix)
    
    
    new_location = where.index[0]
    new_cluster = len_super
    
    #changing values in sub cluster matrix so it can be merged into super cluster 
    

    
    df_sub_linkage_matrix[4] = numpy.NaN
    df_sub_linkage_matrix[5] = numpy.NaN
    

    df_sub_linkage_matrix = prepare_sub_matrix(len_sub, df_sub_linkage_matrix, new_location, new_cluster, current_sub, len_super)
    
    #modify where line and add it to end of sub cluster matrix 
    where.at[new_location, where_col] =  new_location + len_super +  (2 * len_sub) 
    
    print("n_super_samples")
    print(n_super_samples)
            
    if where.loc[new_location, not_where] >= n_super_samples:
            if where.loc[new_location, not_where] - n_super_samples < new_location:
                where.at[new_location, not_where] = where.loc[new_location, not_where] +len_sub
            else:
                where.at[new_location, not_where] = where.loc[new_location, not_where] + (2 * len_sub) 

    where.at[new_location, 3] =  where.loc[new_location, 3] + n_sub_samples
    
     
    print("WHERE")
    print(where)

    #a really dumb way to rename the index
    #.setindex needs a col to set as the index 
    total_sub = pd.DataFrame(where.to_numpy(), index = [where.index[0] + n_sub_samples-1])
    
    total_sub = df_sub_linkage_matrix.append(total_sub)
    
    
    df_super_linkage_matrix = prepare_super(len_super, df_super_linkage_matrix, len_sub, new_location)
    
    
    print("REMOVING")
    print(new_location)
    
    print("TOTAL SUB")
    print(total_sub)
    
    print("NEW LOCATION")
    print(new_location)
    print("N_SUB_SAMPLES")
    print(n_sub_samples)
    print("len_sub")
    print(len_sub)
    print("len_super")
    print(len_super)
    
    

    print("CHANGE TO INDEX")
    print(list(range(0, new_location)) + list(range(new_location + len_sub + 1 , len_super + len_sub - 1)))
    df_super_linkage_matrix.index = list(range(0, new_location)) + list(range(new_location + len_sub + 1 , len_super + len_sub))
    
    
    
    #merge then sort by index 
    new_linkage_matrix = df_super_linkage_matrix.append(total_sub)
    new_linkage_matrix.sort_index(inplace = True)
    
    print(new_linkage_matrix)
    
    total_linkage_matrix = new_linkage_matrix
    
    
    
    

print(new_linkage_matrix)
scipy.cluster.hierarchy.is_valid_linkage(new_linkage_matrix)

In [ ]:
for row in new_linkage_matrix.iterrows():
    print(row)

In [ ]:
#a strange way to get all the values that are not null
no_nan_4 = new_linkage_matrix.loc[new_linkage_matrix[4] > -1]
new_0 = pd.DataFrame(no_nan_4[4])
new_0.columns = [0]
print(new_0)

#a strange way to get all the values that are not null
no_nan_5 = new_linkage_matrix.loc[new_linkage_matrix[5] > -1]
new_1 = pd.DataFrame(no_nan_5[5])
new_1.columns = [1]
print(new_1)


print(new_0.sort_values(0))
print(new_1.sort_values(1))

In [ ]:
new_linkage_matrix.update(new_0)
new_linkage_matrix.update(new_1)
print(new_linkage_matrix)

In [ ]:
new_linkage_matrix = new_linkage_matrix.drop(4, axis = 1)
new_linkage_matrix = new_linkage_matrix.drop(5, axis = 1)
print(new_linkage_matrix)

In [ ]:
#one of two checks that my code often fails, see which row causes the failure 

#_check_hierarchy_uses_cluster_before_formed

Z = numpy.asarray(new_linkage_matrix, order='c')

n = Z.shape[0] + 1
for i in range(0, n - 1):
    if Z[i, 0] >= n + i or Z[i, 1] >= n + i:
        print(i)
        print(Z[i, 0])
    else:
        print(False)

In [ ]:
#one of two checks that my code often fails, see which row causes the failure 

#check_hierarchy_uses_cluster_more_than_once

n = Z.shape[0] + 1
chosen = set([])
for i in range(0, n - 1):
    if (Z[i, 0] in chosen) or (Z[i, 1] in chosen) or Z[i, 0] == Z[i, 1]:
        print(i)
        print(Z[i, 0])
        print(True)
    chosen.add(Z[i, 0])
    chosen.add(Z[i, 1])
    print(False)

print(chosen)

for i in range(0, 2607):
    if i not in chosen:
        print(i)

In [ ]:
for row in new_linkage_matrix.iterrows():
    print(row)

In [ ]:
#new_linkage_matrix.at[354, 1] = 1388.0

In [ ]:
childs = []

for i in range(0, len(new_linkage_matrix)):
    childs.append((int(new_linkage_matrix.loc[i, 0]), int(new_linkage_matrix.loc[i, 1])))



In [ ]:
len(childs)

In [ ]:
 # create the counts of samples under each node
counts = numpy.zeros(len(childs))
print(counts)
n_samples = len(new_linkage_matrix) + 1
for i, merge in enumerate(childs):
    current_count = 0
    for child_idx in merge:
        if child_idx < n_samples:
            current_count += 1  # leaf node
        else:
            current_count += counts[child_idx - n_samples]
    counts[i] = current_count
        
print(counts)

In [ ]:
counts_df = pd.DataFrame(counts)
counts_df.columns = [3]
print(counts_df)

In [ ]:
new_linkage_matrix.update(counts_df)

In [ ]:
print(new_linkage_matrix)

In [ ]:
scipy.cluster.hierarchy.is_valid_linkage(new_linkage_matrix)

In [ ]:
r_dict2 = dendrogram(new_linkage_matrix, truncate_mode='level', p=51, distance_sort = "ascending", ax= axs[2])
plt.show()

In [ ]:
df_r_dict2_labels = r_dict2["leaves"]

In [ ]:
print(r_dict2["leaves"])

In [ ]:
print(df_r_dict2_labels.index(0))

In [ ]:
print(r_dict2["ivl"][0])

super_and_sub_dendro_labels_index = []

for i in range(0, len(r_dict2["ivl"])):
    super_and_sub_dendro_labels_index.append(df_r_dict2_labels.index(i))
    

print(super_and_sub_dendro_labels_index)

In [ ]:
result_df = pd.DataFrame(result)

df_super_and_sub_dendro_labels_index = pd.DataFrame(super_and_sub_dendro_labels_index)
df_super_and_sub_dendro_labels_index.rename(columns = {0:1305}, inplace = True)

sas_transposed_labels_df = df_super_and_sub_dendro_labels_index.transpose()

sas_transposed_labels_df.insert(1305, 1305, 20000)

print(result_df)
print(df_super_and_sub_dendro_labels_index)
print(sas_transposed_labels_df)

sas_column_label_df = result_df.join(df_super_and_sub_dendro_labels_index)

sas_label_df = sas_column_label_df.append(sas_transposed_labels_df, ignore_index=True)

print(sas_label_df)

sas_col_sorted = sas_label_df.sort_values(1305)

sas_sorted_df = sas_col_sorted.sort_values(1305, axis = 1)

print(sas_col_sorted)
print(sas_sorted_df)

with_labels_sas_sorted_df = sas_sorted_df

sas_sorted_df = sas_sorted_df.drop(1305, axis = 0)
sas_sorted_df = sas_sorted_df.drop(1305, axis = 1)

final_output_test = sas_sorted_df.to_numpy()

print(sas_sorted_df)

In [ ]:
fig_super = pltexpr.imshow(sas_sorted_df.to_numpy(), title="graph of sub clusters")
fig_super.show()

In [ ]:
print(with_labels_sas_sorted_df)

In [ ]:
super_cluster_for_sub_labels = []

for index in with_labels_sas_sorted_df.index:
    super_cluster_for_sub_labels.append(with_labels_super_sorted_df[index][len(with_labels_super_sorted_df )-1])
    
df_super_cluster_for_sub_labels = pd.DataFrame(super_cluster_for_sub_labels)

print(df_super_cluster_for_sub_labels)

      

In [ ]:
df_super_cluster_for_sub_labels.rename(columns = {0:1305}, inplace = True)

final_transposed_labels_df = df_super_cluster_for_sub_labels.transpose()


print(sas_sorted_df)
print(df_super_cluster_for_sub_labels)
print(final_transposed_labels_df)

super_cluster_for_sub_labels.pop(1305)

sas_sorted_df_2 = sas_sorted_df

sas_sorted_df_2[1305] = super_cluster_for_sub_labels

print("sas_sorted_df_2")
print(sas_sorted_df_2)

print("final_transposed_labels_df")
print(final_transposed_labels_df)

index2 = list(sas_sorted_df_2.index)
index2.append(1305)

final_transposed_labels_df.columns = index2

final_label_df = sas_sorted_df_2.append(final_transposed_labels_df, ignore_index=True)

print("final_label_df")
print(final_label_df)


final_col_sorted = final_label_df.sort_values(1305)

final_sorted_df = final_col_sorted.sort_values(1305, axis = 1)

print(final_col_sorted)
print(final_sorted_df)

with_labels_final_sorted_df = final_sorted_df

final_sorted_df = final_sorted_df.drop(1305, axis = 0)
final_sorted_df = final_sorted_df.drop(1305, axis = 1)

print(final_sorted_df)

In [ ]:
print(with_labels_final_sorted_df)

In [ ]:
print(super_sorted_df)

In [ ]:
fig_super = pltexpr.imshow(super_sorted_df.to_numpy(), title="graph of grouped data done by using df then reps Hierarchical Clustered")
fig_super.show()

In [ ]:
fig_super_and_sub = pltexpr.imshow(final_sorted_df.to_numpy(), title="graph of sub and super clustered")
fig_super_and_sub.show()

In [ ]:
final_sorted_df.to_csv("final_sorted_df", sep = "\t")

In [ ]:
from scipy.cluster import hierarchy
def getNewick(node, newick, parentdist, leaf_names):
    if node.is_leaf():
        return "%s:%.2f%s" % (leaf_names[node.id], parentdist - node.dist, newick)
    else:
        if len(newick) > 0:
            newick = "):%.2f%s" % (parentdist - node.dist, newick)
        else:
            newick = ");"
        newick = getNewick(node.get_left(), newick, node.dist, leaf_names)
        newick = getNewick(node.get_right(), ",%s" % (newick), node.dist, leaf_names)
        newick = "(%s" % (newick)
        return newick
#from https://stackoverflow.com/questions/28222179/save-dendrogram-to-newick-format/31878514#31878514 

tree = hierarchy.to_tree(new_linkage_matrix,False)
newick = getNewick(tree, "", tree.dist, r_dict2["leaves"])

In [ ]:
flipped_col_final = list(final_sorted_df.columns)
flipped_col_final.reverse()

print(flipped_col_final)

In [ ]:
final_sorted_df_numpy = final_output_test
flipped = numpy.flip(final_sorted_df_numpy)

#final_sorted_df_numpy = final_sorted_df.to_numpy()
#flipped = numpy.flip(final_sorted_df_numpy)

template_file = open("template.json", "r")
json_object = json.load(template_file)
template_file.close()

json_object["columnDendrogram"] = newick
json_object["rowDendrogram"] = newick

dataset = json_object["dataset"]
print(dataset)

dataset["rows"] = len(flipped[0])
dataset["columns"] = len(flipped)

#print("flipped.tolist()")
#print(flipped.tolist()[0])

dataset["seriesArrays"] = [flipped.tolist()]

r_dict2["leaves"].reverse()

print("LABELS")
print(r_dict2["leaves"])

dataset["rowMetadataModel"]["vectors"][0]["array"] = r_dict2["leaves"]
dataset["columnMetadataModel"]["vectors"][0]["array"] = r_dict2["leaves"]    

dataset["rowMetadataModel"]["vectors"][1]["array"] = r_dict2["leaves"]
dataset["columnMetadataModel"]["vectors"][1]["array"] = r_dict2["leaves"]    



output_file = open("super_and_sub_file.json", "w")
json.dump(json_object, output_file)
output_file.close()

In [ ]:
flipped_df = pd.DataFrame(flipped)
flipped_df.columns = r_dict2["leaves"]
flipped_df.index = r_dict2["leaves"]


print(flipped_df)

In [ ]:
import random

In [ ]:
for i in range(0, len(flipped_df)):
    r_0_1305 = random.randint(0, len(flipped_df)-1)
    print(result_df.loc[i, r_0_1305] == flipped_df.loc[i, r_0_1305])

In [ ]:

print(sas_sorted_df)


for i in range(0, len(flipped_df)):
    r_0_1305 = random.randint(0, len(flipped_df)-1)
    print(result_df.loc[i, r_0_1305] == sas_sorted_df.loc[i, r_0_1305])



In [ ]:
print(result_df)